# Calcul de l'autonomie réelle

In [ ]:
import pandas as pd
from core.sql_utils import get_connection
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from core.s3.s3_utils import S3Service
from core.s3.settings import S3Settings
from core.spark_utils import create_spark_session
from sqlalchemy import text
from pyspark.sql.functions import col
from plotly.subplots import make_subplots
import plotly.figure_factory as ff
import pyspark.sql.functions as F

settings = S3Settings()

spark = create_spark_session(
    settings.S3_KEY,
    settings.S3_SECRET
)

s3 = S3Service()

In [ ]:
from core.sql_utils import get_sqlalchemy_engine
engine = get_sqlalchemy_engine()
con = engine.connect()
dbeaver_df = pd.read_sql(text("""SELECT v.vin,  model_name, type, version, autonomy, MIN(vd.soh) as soh FROM vehicle_data vd
                              JOIN vehicle v ON vd.vehicle_id = v.id
                              JOIN vehicle_model vm ON v.vehicle_model_id = vm.id
                              GROUP BY v.vin, model_name, type, version, autonomy; """), con)



## 1. Chargement des données 

In [ ]:
result_phases = s3.read_parquet_df_spark(spark, "result_phases/result_phases_tesla_fleet_telemetry.parquet")


## 2. Filtrage des données

- vehicule avec plus de 500 km suivis
- Prendre une décharge de minimum 5% (test jusqu`à 10%)
- Retirer les valeurs d'odomètre sur la décharge supérieur à 130% de WLTP et qu'il ai fait au moins 5 km

### 2.1. Filtre sur les véhicules
Pour valider le calcul on veut les véhicules qu'on a suivi au minimum pendant 500 km 

In [ ]:
def filter_valid_vins(pph, min_odometer_diff=500):
    vin_stats = (
        pph.groupBy("VIN")
        .agg(
            F.min("ODOMETER_FIRST").alias("odometer_start"),
            F.max("ODOMETER_LAST").alias("odometer_end"),
        )
        .withColumn(
            "odometer_diff", F.col("odometer_end") - F.col("odometer_start")
        )
    )

    valid_vins = vin_stats.filter(F.col("odometer_diff") > min_odometer_diff)
    df_filtered = pph.join(valid_vins.select("VIN"), "VIN", "inner")

    return df_filtered

In [ ]:
df_result = filter_valid_vins(result_phases)

In [ ]:
df_result_pd = df_result.filter(F.col("SOC_DIFF") < 0).select("VIN", "MODEL", "VERSION", "BATTERY_NET_CAPACITY", "SOC_DIFF",  "ODOMETER_DIFF", "RATIO_KM_SOC", "ODOMETER_FIRST", "ODOMETER_LAST", "SOC_FIRST", "SOC_LAST").dropna(subset=['RATIO_KM_SOC']).toPandas()


In [ ]:
df_result_pd = df_result_pd.merge(dbeaver_df.dropna()[['vin', 'autonomy', 'soh', "version"]], left_on='VIN', right_on='vin', how='left')

In [ ]:
df_result_pd

### 2.2. Filtre sur ODOMETER_DIFF

- Minimum 5km de décharge
- Pas de décharge supérieur à 130 % WLTP

In [ ]:
df_result_pd["ODOMETER_CATEGORY"] = pd.cut(
    df_result_pd["ODOMETER_DIFF"],
    bins=[0, 5,10, 25, 50, 100, 200, 300, 500, float('inf')],
    labels=["<5 km", "5-10 km", "10-25 km", "25-50 km", "50-100 km", "100-200 km", "200-300 km", "300-500 km", "≥500 km"],
    right=False
)

df_result_pd['ODOMETER_CATEGORY'].value_counts()



In [ ]:
px.scatter(df_result_pd, x='ODOMETER_DIFF', y='RATIO_KM_SOC',)

### 2.3. Filtre sur SOC_DIFF

- Prendre le SoC > 5% (test jusqu`à 10%)

In [ ]:
df_result_pd["SOC_DIFF_CATEGORY"] = pd.cut(
    df_result_pd["SOC_DIFF"], 
    bins=[float('-inf'), -75, -50, -25, -10, -5, 0],
    labels=[">75%", "75-50%", "50-25%", "25-10%", "10-5%", "5-0%"], 
    right=False
)

df_result_pd['SOC_DIFF_CATEGORY'].value_counts()

In [ ]:
px.scatter(df_result_pd, x='SOC_DIFF', y='RATIO_KM_SOC',)

In [ ]:
soc_diff_min = 5 
odometer_diff_min = 5
wltp_ratio_max = 1.3
df_result_filter = df_result_pd[
    (df_result_pd['SOC_DIFF'] < - soc_diff_min) & 
    (
        df_result_pd['autonomy'].isna() |
        (
            (df_result_pd['ODOMETER_DIFF'] < df_result_pd['autonomy'] * wltp_ratio_max)
        )
    )
    & (df_result_pd['ODOMETER_DIFF'] > odometer_diff_min)
]


## 3. Évaluation et analyse du ratio_km_soc

Cette section évalue la qualité et la distribution du ratio `ratio_km_soc`, qui représente le nombre de kilomètres parcourus par pourcent de SOC consommé.

Je pense que filtrer les `ratio_km_soc > 10` est une première solution pour éviter les valeurs absurde (pas d'autonomie WLTP > 1000km dans tous les cas).


### 3.1 application du filtre RATIO_KM_SOC < 10


In [ ]:
df_result_filter.sort_values(by="RATIO_KM_SOC", ascending=False).head(10)

In [ ]:
df_result_cut = df_result_filter[df_result_filter['RATIO_KM_SOC'] < 10]

In [ ]:
df_result_cut.dtypes

In [ ]:
# Calcul de la corrélation avec RATIO_KM_SOC uniquement
corr_data = df_result_cut[['BATTERY_NET_CAPACITY', 'SOC_DIFF', 'ODOMETER_DIFF', 'RATIO_KM_SOC', 'SOC_FIRST', 'SOC_LAST', 'ODOMETER_FIRST', 'ODOMETER_LAST']].corr()
corr_ratio = corr_data['RATIO_KM_SOC'].drop('RATIO_KM_SOC').sort_values(key=abs, ascending=False)

# Visualisation en barres horizontales pour plus de lisibilité
fig = px.bar(
    x=corr_ratio.values,
    y=corr_ratio.index,
    orientation='h',
    title='Corrélation avec RATIO_KM_SOC',
    labels={'x': 'Coefficient de corrélation', 'y': 'Variable'},
    text=[f'{val:.3f}' for val in corr_ratio.values]
)
fig.update_traces(textposition='outside')
fig.update_layout(height=400, showlegend=False)
fig.show()

### 3.1 Étude sur le % de décharge (SOC_DIFF)

Analyse de l'impact du pourcentage de décharge sur le ratio_km_soc.

In [ ]:
px.scatter(df_result_cut, x='SOC_DIFF', y='RATIO_KM_SOC', color='MODEL')

In [ ]:
# Calculer le coefficient de variation (CV = std/mean) par catégorie de SOC_DIFF
cv_by_soc = df_result_cut.groupby('SOC_DIFF_CATEGORY')['RATIO_KM_SOC'].agg([
    lambda x: x.std() / x.mean() if x.mean() != 0 else np.nan,
    'count'
]).round(4)
cv_by_soc.columns = ['CV', 'count']

# Visualisation
fig = px.bar(
    cv_by_soc.reset_index(),
    x='SOC_DIFF_CATEGORY',
    y='CV',
    title="Coefficient de variation du ratio_km_soc par catégorie de SOC_DIFF",
    labels={'SOC_DIFF_CATEGORY': 'Catégorie de décharge (%)', 'CV': 'Coefficient de variation'}
)
fig.update_layout(xaxis_tickangle=-45, height=500)
fig.show()

# Analyse de la variance par modèle et SOC_DIFF
variance_analysis = df_result_cut.groupby(['MODEL', 'SOC_DIFF_CATEGORY'])['RATIO_KM_SOC'].agg([
    'count', 'std', 'mean'
]).round(2)


### 3.2 Étude sur le kilométrage (ODOMETER_DIFF)

Analyse de l'impact de la distance parcourue sur le ratio_km_soc.

In [ ]:
px.scatter(df_result_cut, x='ODOMETER_DIFF', y='RATIO_KM_SOC', color='MODEL')

In [ ]:
# Calculer le coefficient de variation (CV = std/mean) par catégorie de SOC_DIFF
cv_by_soc = df_result_cut.groupby('ODOMETER_CATEGORY')['RATIO_KM_SOC'].agg([
    lambda x: x.std() / x.mean() if x.mean() != 0 else np.nan,
    'count'
]).round(4)
cv_by_soc.columns = ['CV', 'count']

# Visualisation
fig = px.bar(
    cv_by_soc.reset_index(),
    x='ODOMETER_CATEGORY',
    y='CV',
    title="Coefficient de variation du ratio_km_soc par ODOMETER_CATEGORY",
)
fig.update_layout(xaxis_tickangle=-45, height=500)
fig.show()

# Analyse de la variance par modèle et SOC_DIFF
variance_analysis = df_result_cut.groupby(['MODEL', 'ODOMETER_CATEGORY'])['RATIO_KM_SOC'].agg([
    'count', 'std', 'mean'
]).round(2)


## 4. Autonomie réelle par modèle

Calcul de l'autonomie réelle normalisée à 100% SOH pour chaque modèle/version.

### 4.1. Méthodologie de calcul

**Objectif**: Calculer l'autonomie réelle normalisée à 100% de SOH

**Formule**: 
```
autonomie_reelle_100 = ratio_km_soc * 100 * (1 / SOH)
```

Où:
- `ratio_km_soc`: nombre de km parcourus par % de SOC
- `100`: pour obtenir l'autonomie totale (0% à 100%)
- `1 / SOH`: facteur de normalisation pour ramener à une batterie neuve (100% SOH)


In [ ]:
df_result_cut['REAL_AUTONOMY'] = (
    df_result_cut['RATIO_KM_SOC'] * 100 
)

In [ ]:
df_result_cut['REAL_AUTONOMY_100'] = (
    df_result_cut['REAL_AUTONOMY'] * (1 / df_result_cut['soh'].replace(0, np.nan).replace(np.nan, 1)))


In [ ]:
df_result_cut

In [ ]:
df_result_cut

In [ ]:
px.scatter(df_result_cut, x='REAL_AUTONOMY_100', y='soh', color='version')

In [ ]:
df_result_cut.groupby(['MODEL', 'VERSION', 'version']).agg({'REAL_AUTONOMY_100': ['mean', 'median', 'max', 'min'], 'autonomy': 'max'}).reset_index()